This notebook walks through how to make graphs in this repo. You can copy this example.ipynb and rename it for your own graphing work. <b> Please do each graph in a seperate notebook!</b>

1. Import the appropriate packages. We'll use plotly.graph_objects for making all of our graphs.

In [1]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

2. We'll get data using the fredapi package. Set the path to the text file with your API key

In [2]:
# Call the graph the exact same thing as its notebook (minus the ipynb suffix)
GRAPH_NAME = "un_race"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

3. Set the fed_2025 template as default

In [3]:
pio.templates.default = 'fed_2025'

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

4. Use the fredapi to get the data and prepare it for graphing. Documentation on the optional parameters that can be passed to the get_series called are found here (the documentation in fredapi is out of date). 

https://fred.stlouisfed.org/docs/api/fred/series_observations.html#Description

If you get data from somewhere else thats fine too! Put the raw csv in the "raw_data" folder and read it in here. Make sure not to edit the raw data, just transform and graph it.

In [4]:
# The get_series_info method may be useful
fred.get_series_info(series_id="CPIAUCSL")

# It's good practice to store the series codes in a dictionary with their names
series_codes = {
    "African American": "LNS14000006",
    "16-24 Year Olds":"LNS14024887",
    "Overall": "UNRATE",
}

In [11]:
today = dt.date.today()

un_black=fred.get_series(
    series_id=series_codes["African American"],
    observation_start=dt.date(2023, 1, 1),
    observation_end=today,
    frequency='m',
).rename("Black")

un_white=fred.get_series(
    series_id=series_codes["16-24 Year Olds"],
    observation_start=dt.date(2023, 1, 1),
    observation_end=today,
    frequency='m'
).rename("16-24 Year Olds")

un_asian=fred.get_series(
    series_id=series_codes["Overall"],
    observation_start=dt.date(2023, 1, 1),
    observation_end=today,
    frequency='m'
).rename("Overall")

# un_latino=fred.get_series(
#     series_id=series_codes["Hispanic or Latino"],
#     observation_start=dt.date(2019, 1, 1),
#     observation_end=today,
#     frequency='m'
# ).rename("Hispanic or Latino")

joined_df = pd.concat(
    [un_black, un_white, un_asian], #un_latino], # srs to join
    axis=1, # multiple observations per index entry
    join='inner' # only include observations where none of the 3 are missing
)

# Naming the index makes it easier tot title
joined_df.index.name = "Date"

joined_df.tail()

,Black,16-24 Year Olds,Overall
Date,,,
2025-04-01,6.3,9.6,4.2
2025-05-01,6.0,9.7,4.2
2025-06-01,6.8,10.0,4.1
2025-07-01,7.2,10.0,4.2
2025-08-01,7.5,10.5,4.3


5. Now that all our data is ready, make the graph and have it save itself as a .html file to graph_output whenver the notebooks is rerun. The name of the file should exactly match the notebook name. For instance, this file "example.ipynb" produces the graph "example.html." Nice work, you made a graph! 

In [16]:
# First make the figure
fig = go.Figure()

# Loop the columns of the dataframe and plot each as a separate trace
for col in joined_df.columns:
    fig.add_trace(
        go.Scatter(
            x=joined_df.index,
            y=joined_df[col],
            mode='lines',
            name=col
        )
    )

# Update the titles, using the html tage <sup> for a subtitle 
fig.update_layout(
    title = dict(text = 'Unemployment Rates<br><sup>Monthly, Seasonally Adjusted </sup>'),
    xaxis_title="Date",
    yaxis_title="Share of Labor Force (%)",
    height=500,
    width=800,
    legend=dict(
        align="v",
        font=dict(size=18))
)


# This is graph specific, but here we want the y-axis to be percent signs 
fig.update_yaxes(
    tickformat="0.2f%",
    ticksuffix="%"
)

# Again, graph specific, we have a mutliyear series and want tick marks to be years
fig.update_xaxes(
    type='date',
    tickformat='%b %Y',
)

ut.add_end_labels(fig = fig)

# Show our figure (Dimensions may be off on different screen sizes)
fig.show()

# This should be the same for EVERY GRAPH!
# Save it to the graph_output folder with the name matching the file, as HTML
fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")

ValueError: Invalid property specified for object of type plotly.graph_objs.layout.Legend: 'align'

Did you mean "valign"?

    Valid properties:
        bgcolor
            Sets the legend background color. Defaults to
            `layout.paper_bgcolor`.
        bordercolor
            Sets the color of the border enclosing the legend.
        borderwidth
            Sets the width (in px) of the border enclosing the
            legend.
        entrywidth
            Sets the width (in px or fraction) of the legend. Use 0
            to size the entry based on the text width, when
            `entrywidthmode` is set to "pixels".
        entrywidthmode
            Determines what entrywidth means.
        font
            Sets the font used to text the legend items.
        groupclick
            Determines the behavior on legend group item click.
            "toggleitem" toggles the visibility of the individual
            item clicked on the graph. "togglegroup" toggles the
         